# Análise de Banco de Dados com Geração de Reports em PDF

Criação e análise de um banco de dados fictício para obter graficamente dados referentes a vendas por segmentos

# Imports

In [1]:
import sqlite3
import random
import datetime
import kaleido

# Comandos SQL para criar a tabela

In [2]:
create_sales_table = '''CREATE TABLE vendas (
                        id_venda INTEGER PRIMARY KEY,
                        data_venda DATE,
                        id_cliente INTEGER,
                        id_produto INTEGER,
                        quantidade INTEGER,
                        valor_unitario DECIMAL(10, 2),
                        valor_total DECIMAL(10, 2),
                        FOREIGN KEY (id_cliente) REFERENCES clientes(id_cliente),
                        FOREIGN KEY (id_produto) REFERENCES produtos(id_produto)
                     )'''

create_products_table = '''CREATE TABLE produtos (
                            id_produto INTEGER PRIMARY KEY,
                            nome_produto TEXT,
                            valor_unitario DECIMAL(10, 2)
                         )'''

create_customers_table = '''CREATE TABLE clientes (
                            id_cliente INTEGER PRIMARY KEY,
                            nome TEXT,
                            sobrenome TEXT,
                            email TEXT,
                            telefone TEXT
                         )'''

# Comandos SQL para inserir amostra de dados nas tabelas

In [3]:
insert_produtos_data = '''INSERT INTO produtos (nome_produto, valor_unitario) VALUES (?, ?)'''

insert_vendas_data = '''INSERT INTO vendas (data_venda, id_cliente, id_produto, quantidade, valor_unitario, valor_total) VALUES (?, ?, ?, ?, ?, ?)'''

insert_clientes_data = '''INSERT INTO clientes (nome, sobrenome, email, telefone) VALUES (?, ?, ?, ?)'''

# Define amostra de dados para as tabelas produtos e clientes

In [4]:
produtos = [('Produto A', 50.00), ('Produto B', 25.00), ('Produto C', 75.00), ('Produto D', 40.00), ('Produto E', 60.00)]

clientes = [
    ('James', 'Hetfield', 'hetfield@example.com', '92555-1234'),
    ('David', 'Ghrol', 'dghrol@example.com', '97225-5678'),
    ('Bruce', 'Dickinson', 'dickinsonb@example.com', '98248-9012'),
    ('Ozzy', 'Osbourne', 'o_osbourne@example.com', '94317-3456'),
    ('Steven', 'Tyler', 'steventyler@example.com', '95571-7890'),
    ('Brian', 'Johnson', 'b_johnson@example.com', '93158-2345'),
    ('Dolores', 'O Riodon', 'doloresor@example.com', '94786-6789'),
    ('Linda', 'Perry', 'perry_linda@example.com', '95789-1234'),
    ('Tarja', 'Turunen', 'tarjaturunen@example.com', '99781-5678'),
    ('Lzzy', 'Halle', 'lzzyhalle@example.com', '99324-9012')
]

# Define datas de início e fim para geração de dados de vendas

In [5]:
data_inicio = datetime.date(2022, 1, 1)
data_final = datetime.date(2022, 12, 31)

# Conecta com o database e cria as tabelas

In [6]:
with sqlite3.connect('vendas.db') as conn:
    # Cria a tabela vendas
    conn.execute(create_sales_table)

    # Cria a tabela produtos
    conn.execute(create_products_table)

    # Cria a tabela clientes
    conn.execute(create_customers_table)

    # Insere amostra de dados na tabela produtos
    for produto in produtos:
        conn.execute(insert_produtos_data, produto)

    # Insere amostra de dados na tabela clientes
    for cliente in clientes:
                conn.execute(insert_clientes_data, cliente)

    # Insere amostra de dados na tabela vendas
    for i in range(1000):
        venda_data = data_inicio + datetime.timedelta(days=random.randint(0, 364))
        id_cliente = random.randint(1, len(clientes))
        id_produto = random.randint(1, len(produtos))
        quantidade = random.randint(1, 10)
        valor_unitario = produtos[id_produto-1][1]
        valor_total = quantidade * valor_unitario
        conn.execute(insert_vendas_data, (venda_data, id_cliente, id_produto, quantidade, valor_unitario, valor_total))

    # Faz o commit 
    conn.commit()

    print("Banco de dados criado com sucesso!")

Banco de dados criado com sucesso!


# Sales Report using SQL + Pandas

Depenências

In [8]:
from datetime import date
from pathlib import Path
import sqlite3

import pandas as pd  
import plotly.express as px  
from fpdf import FPDF  

# Definição de caminhos (paths) e estilo de gráficos

In [9]:
# Define o template do plotly
# Outros examplos: "plotly", "ggplot2", "seaborn", "simple_white", "plotly_dark", "plotly_white", ..
plotly_template = "presentation"

In [10]:
# Definição dos paths
current_dir = Path(__file__).parent if "__file__" in locals() else Path.cwd()
database_path = current_dir / "vendas.db"
output_dir = current_dir / "output"

# Cria o diretório de output e seu directorio parente caso ainda não existam
output_dir.mkdir(parents=True, exist_ok=True)

# Total Sales by Month

In [11]:
# Cria a conexão com o database
conn = sqlite3.connect(database_path)

In [12]:
# Executa a query e carrega os resultados em um DataFrame do Pandas
query = '''
SELECT data_venda, SUM(valor_total) as venda_total
FROM vendas
GROUP BY data_venda
ORDER BY data_venda ASC
'''
df = pd.read_sql_query(query, conn)

In [13]:
# Display do DF
df

,data_venda,venda_total
0,2022-01-01,1275
1,2022-01-02,250
2,2022-01-03,500
3,2022-01-04,1535
4,2022-01-06,1575
...,...,...
337,2022-12-27,600
338,2022-12-28,1200
339,2022-12-29,1660
340,2022-12-30,450


In [14]:
# Informações dos dados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 342 entries, 0 to 341
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   data_venda   342 non-null    object
 1   venda_total  342 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.5+ KB


In [15]:
# Converte coluna data_venda para datetime
df['data_venda'] = pd.to_datetime(df['data_venda'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 342 entries, 0 to 341
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   data_venda   342 non-null    datetime64[ns]
 1   venda_total  342 non-null    int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 5.5 KB


In [16]:
# Transformar a coluna data_venda em index
df = df.set_index('data_venda')
df.head(3)

,venda_total
data_venda,
2022-01-01,1275
2022-01-02,250
2022-01-03,500


In [17]:
# Reamostrar os dados para uma frequência mensal e calcular a soma
df_mensal = df.resample('M').sum()
df_mensal

,venda_total
data_venda,
2022-01-31,21905
2022-02-28,24490
2022-03-31,23380
2022-04-30,26895
2022-05-31,23285
2022-06-30,22990
2022-07-31,21450
2022-08-31,25590
2022-09-30,20065


In [18]:
# Mapeia o número do mês para o nome curto do mês
df_mensal['mes'] = df_mensal.index.strftime('%b')
df_mensal

,venda_total,mes
data_venda,,
2022-01-31,21905,Jan
2022-02-28,24490,Feb
2022-03-31,23380,Mar
2022-04-30,26895,Apr
2022-05-31,23285,May
2022-06-30,22990,Jun
2022-07-31,21450,Jul
2022-08-31,25590,Aug
2022-09-30,20065,Sep


In [19]:
# Crie a figura Plotly com parâmetro de texto
fig = px.bar(df_mensal,
             x='mes',
             y='venda_total',
             template=plotly_template,
             text='venda_total')

# Configura o layout
fig.update_layout(
    title='Total de Vendas por Mês',
    xaxis_title='mes',
    yaxis_title='Total de Vendas (R$)',
    yaxis_tickprefix='R$',
)

fig.show()

# Salvar o gráfico como uma imagem PNG

In [20]:
fig.write_image(output_dir / 'vendas_mensais.png',
                width=1200,
                height=400,
                scale=4)

# Total de Vendas Por Produto

In [21]:
# Query para carregar os resultados em um DataFrame do Pandas
query = '''
SELECT p.nome_produto, SUM(v.valor_total) as venda_total
FROM vendas v
JOIN produtos p ON v.id_produto = p.id_produto
GROUP BY p.nome_produto
'''
df = pd.read_sql_query(query, conn)

In [22]:
df

,nome_produto,venda_total
0,Produto A,56300
1,Produto B,27475
2,Produto C,89325
3,Produto D,45400
4,Produto E,63720


In [23]:
# Plot dos dados
fig = px.bar(df,
             x='nome_produto',
             y='venda_total',
             template=plotly_template,
             text='venda_total')

fig.update_layout(
    title='Total De Vendas Por Produto',
    xaxis_title='Produto',
    yaxis_title='Total de vendas (R$)',
    yaxis_tickprefix='R$',
)

fig.show()


In [24]:
fig.write_image(output_dir / 'vendas_por_produtos.png',
                width=1200,
                height=400,
                scale=4)

# Top Clientes Por Vendas

In [25]:
query = '''
SELECT c.nome || ' ' || c.sobrenome as nome_do_cliente, SUM(v.valor_total) as venda_total
FROM vendas v
JOIN clientes c ON v.id_cliente = c.id_cliente
GROUP BY nome
ORDER BY venda_total DESC
LIMIT 10
'''
df = pd.read_sql_query(query, conn)

In [26]:
df

,nome_do_cliente,venda_total
0,Tarja Turunen,30775
1,James Hetfield,30590
2,Ozzy Osbourne,30025
3,David Ghrol,29975
4,Bruce Dickinson,28390
5,Linda Perry,28085
6,Steven Tyler,27565
7,Dolores O Riodon,27110
8,Brian Johnson,25860
9,Lzzy Halle,23845


In [27]:
# Plot do Gráfico
fig = px.bar(df,
             x='nome_do_cliente',
             y='venda_total',
             template=plotly_template,
             text='venda_total')

fig.update_layout(
    title='Top Clientes Por Vendas',
    xaxis_title='Cliente',
    yaxis_title='Total de Vendas (R$)',
    yaxis_tickprefix='R$',
)

fig.show()

In [28]:
# Salvar o gráfico como uma imagem PNG
fig.write_image(output_dir / 'vendas_por_clientes.png',
                width=1200,
                height=400,
                scale=4)

# Create PDF Report

In [29]:
# Define cor da font com valores RGB (dark gray)
font_color = (64, 64, 64)

# Encontra todos arquivos PNG na pasta output
chart_filenames = [str(chart_path) for chart_path in output_dir.glob("*.png")]

# Cria um documento PDF e configura o tamanho da da página
pdf = FPDF()
pdf.add_page()
pdf.set_font('Arial', 'B', 24)

# Adicione o título geral da página
title = f"Relatório de vendas a partir de {date.today().strftime('%m/%d/%Y')}"
pdf.set_text_color(*font_color)
pdf.cell(0, 20, title, align='C', ln=1)

# Adicione cada gráfico ao documento PDF
for chart_filename in chart_filenames:
    pdf.ln(10)  # Adicionar preenchimento no topo do próximo gráfico
    pdf.image(chart_filename, x=None, y=None, w=pdf.w - 20, h=0)

# Salva o documento PDF em um arquivo no disco
pdf.output(output_dir / "report_de_vendas.pdf", "F")

''

# Segmento de clientes usando SQL

In [30]:
query = '''
SELECT 
  clientes.id_cliente, 
  clientes.nome || ' ' || clientes.sobrenome as nome_do_cliente, 
  SUM(vendas.valor_total) as venda_total,
  CASE 
    WHEN SUM(vendas.valor_total) > 30000 THEN 'Valor Alto'
    WHEN SUM(vendas.valor_total) > 26000 THEN 'Valor Médio'
    ELSE 'Valor Baixo'
  END as segmento_de_vendas
FROM vendas
INNER JOIN clientes ON vendas.id_cliente = clientes.id_cliente
GROUP BY clientes.id_cliente
ORDER BY venda_total DESC
'''

df = pd.read_sql_query(query, conn)

In [31]:
df

,id_cliente,nome_do_cliente,venda_total,segmento_de_vendas
0,9,Tarja Turunen,30775,Valor Alto
1,1,James Hetfield,30590,Valor Alto
2,4,Ozzy Osbourne,30025,Valor Alto
3,2,David Ghrol,29975,Valor Médio
4,3,Bruce Dickinson,28390,Valor Médio
5,8,Linda Perry,28085,Valor Médio
6,5,Steven Tyler,27565,Valor Médio
7,7,Dolores O Riodon,27110,Valor Médio
8,6,Brian Johnson,25860,Valor Baixo
9,10,Lzzy Halle,23845,Valor Baixo


# Segmento de clientes usando SQL + Pandas

In [32]:
query = '''
SELECT 
  clientes.id_cliente, 
  clientes.nome || ' ' || clientes.sobrenome as nome_do_cliente, 
  SUM(vendas.valor_total) as venda_total
FROM vendas
INNER JOIN clientes ON vendas.id_cliente = clientes.id_cliente
GROUP BY clientes.id_cliente
'''

df = pd.read_sql_query(query, conn)

In [33]:
df

,id_cliente,nome_do_cliente,venda_total
0,1,James Hetfield,30590
1,2,David Ghrol,29975
2,3,Bruce Dickinson,28390
3,4,Ozzy Osbourne,30025
4,5,Steven Tyler,27565
5,6,Brian Johnson,25860
6,7,Dolores O Riodon,27110
7,8,Linda Perry,28085
8,9,Tarja Turunen,30775
9,10,Lzzy Halle,23845


In [34]:
# Agrupa dados por segmento de cliente
bins = [0, 26000, 30000, float('inf')]
labels = ['Valor Baixo', 'Valor Médio', 'Valor Alto']
df['segmento_por_cliente'] = pd.cut(df['venda_total'], bins=bins, labels=labels)

# Ordena dados por total de vendas
df = df.sort_values(by='venda_total', ascending=False)
df

,id_cliente,nome_do_cliente,venda_total,segmento_por_cliente
8,9,Tarja Turunen,30775,Valor Alto
0,1,James Hetfield,30590,Valor Alto
3,4,Ozzy Osbourne,30025,Valor Alto
1,2,David Ghrol,29975,Valor Médio
2,3,Bruce Dickinson,28390,Valor Médio
7,8,Linda Perry,28085,Valor Médio
4,5,Steven Tyler,27565,Valor Médio
6,7,Dolores O Riodon,27110,Valor Médio
5,6,Brian Johnson,25860,Valor Baixo
9,10,Lzzy Halle,23845,Valor Baixo


In [35]:
# Fecha a conexão
conn.close()